In [1]:
import pandas as pd
import glob
from pathlib import Path
from datetime import datetime, date, time 
import calendar
from IPython.display import display, HTML
import re

In [2]:
column_names = [
'Date',
'Year',
'Month',
'Week',
'Model',
'Model_asin',
'Type',
'Classify',
'Campaign Name',
'Imp',
'Clicks',
'Spend',
'Total Sales',
'SKU Units',
'SKU Sales',
'Total Orders'
]
#读取数据

ASIN_FILE_PATH = r'F:\Amazon\SSPA\Campagin\row data\asin.xlsx'
#匹配型号，理解为引射表

AD_TYPE_SB = 'SB'
AD_TYPE_SD = 'SD'
AD_TYPE_SP = 'SP'
file_dict = {
    AD_TYPE_SB:r'F:\Amazon\SSPA\Campagin\row data\SB',
    AD_TYPE_SD:r'F:\Amazon\SSPA\Campagin\row data\SD',
    AD_TYPE_SP:r'F:\Amazon\SSPA\Campagin\row data\SP-Advertised'
}


In [3]:
all_month_end_days = pd.date_range(start='2022-12-01',end='2023-12-31',freq='M')

In [4]:
all_month_end_days = pd.date_range(start='2022-12-01',end='2023-12-31',freq='M')
m_days = {}
#定义月底日期
for month_end_day in all_month_end_days:
    days = set()
    month_first_day = month_end_day.replace(day=1)
    days.add(month_first_day)
    sun_days = pd.date_range(start=month_first_day,end=month_end_day, freq='W-SUN')
    for sun_day in sun_days:
        days.add(sun_day)
    days.add(month_end_day)
    m_days[month_first_day] = sorted(days)
#定义week of month
week_of_month_dict = {}
for m, days in m_days.items() :
    idx = 0
    week_count = 1
    days_list = list(days)
    while(idx + 1 < len(days)):
        ds = pd.date_range(start=days_list[idx],end=days_list[idx+1], freq='D')
        for d in ds:
            week_of_month_dict[d] = week_count
        week_count = week_count + 1
        idx = idx + 1
        
today_beginning = datetime.combine(date.today(), time()) 

In [5]:
asin_df = pd.read_excel(ASIN_FILE_PATH)
asin_dict = asin_df.set_index(['Advertised ASIN','Advertised SKU'])['Model'].to_dict()

In [6]:
# asin_dict

In [7]:
asin_not_found_set = set()

In [8]:
def read_files(ad_file_path):
    files = Path(ad_file_path).glob('*.xlsx')
    dfs = list()
    for i, f in enumerate(files):
        print(f.name)
        data = pd.read_excel(f)
        data['file'] = f'File {i}'
        dfs.append(data)
    df = pd.concat(dfs, ignore_index=True)
    return df

def week_of_month(date):
    ts = date.to_pydatetime()
    wom = week_of_month_dict[ts]
    return wom

#提取SP广告报表的Model
def extract_sp_model(cn):
    model_candidate = cn.split("_")[0]
    if model_candidate == 'RR':
        return 'Others-sales'
    elif model_candidate == 'S7series&S7MaxVseries':
        return 'S7&S7MaxVseries'
    elif model_candidate == 'S7series&S7Maxvseries':
        return 'S7&S7MaxVseries'
    elif model_candidate == 'Accessory':
        return 'Others-sales'
    elif model_candidate == 'Allmodel':
        return 'Others-sales'
    elif model_candidate == 'All':
        return 'Others-sales'
    elif model_candidate == 'S7MaxV Ultra':
        return 'S7MaxV++'
    elif model_candidate == 'S7 MaxV Ultra':
        return 'S7MaxV++'
    elif model_candidate == 'E5mop':
        return 'E5Mop'
    elif model_candidate == 'S7Maxv ultra':
        return 'S7MaxV++'
    elif model_candidate == 'S7Maxv+':
        return 'S7MaxV+'    
    elif model_candidate == 'Q7+Q7Max+':
        return 'Q7Max+'
    return model_candidate

#提取SP广告报表的Class
def extract_sp_class1(cn):
    if "SD" in cn:
        return "SD"
    return "SP"

#提取Sb广告报表的Model
def extract_sp_model_sb(cn):
    end_wit_sb = re.search('SB-?(?![A-Z])', cn)
#     end_wit_sb = cn.endswith('SB')
    if end_wit_sb:
        return 'Others-ads'
    else:
        return extract_sp_model(cn)
    
#提取SbV广告报表的Class
def extract_sbv_class(cn):
#     end_wit_sb = re.search('SBV-?(?![A-Z])', cn)
#     end_wit_sbv = cn.endswith('SBV')
    end_wit_sbv = 'SBV' in cn
    if end_wit_sbv:
        return 'SBV'
    else:
        return 'SB'

def extract_sp_model_asin(aa):
    return asin_dict[aa]

def extract_sp_model_asin2(aa,sku):
    asin_sku_key = (aa,sku)
    model = asin_dict.get(asin_sku_key)
    if model is None:
        print("{} is empty".format(aa))
        not_foun_asin = (aa,sku)
        asin_not_found_set.add(not_foun_asin)
    return model

def extract_excel(ad_type,ad_file_path):
    print(ad_type)
    load_excel_files_df = read_files(ad_file_path)
    if AD_TYPE_SB is ad_type:
        load_excel_files_df['Year'] = load_excel_files_df['Date'].dt.year
        load_excel_files_df['Month'] = load_excel_files_df['Date'].dt.month
        load_excel_files_df['Week'] = load_excel_files_df['Date'].apply(week_of_month)
        load_excel_files_df['Model'] = load_excel_files_df['Campaign Name'].apply(extract_sp_model_sb)
        load_excel_files_df['Model_asin'] = load_excel_files_df['Model']
        load_excel_files_df['Type'] = load_excel_files_df['Campaign Name'].str.split(pat="_").str[-2]
        load_excel_files_df['Classify'] = load_excel_files_df['Campaign Name'].apply(extract_sbv_class)
        load_excel_files_df['Imp'] = load_excel_files_df['Impressions']
        load_excel_files_df['Total Sales'] = load_excel_files_df['14 Day Total Sales '].round(2)
        load_excel_files_df['SKU Units'] = 0
        load_excel_files_df['SKU Sales'] = 0
        load_excel_files_df['Total Orders'] = load_excel_files_df['14 Day Total Orders (#)']
        
#     if AD_TYPE_SBV is ad_type or AD_TYPE_SD is ad_type:
    if AD_TYPE_SD is ad_type:
        load_excel_files_df['Year'] = load_excel_files_df['Date'].dt.year
        load_excel_files_df['Month'] = load_excel_files_df['Date'].dt.month
        load_excel_files_df['Week'] = load_excel_files_df['Date'].apply(week_of_month)
        load_excel_files_df['Model'] = load_excel_files_df['Campaign Name'].apply(extract_sp_model)
        load_excel_files_df['Model_asin'] = load_excel_files_df['Model'].apply(extract_sp_model)                               
        load_excel_files_df['Type'] = load_excel_files_df['Campaign Name'].str.split(pat="_").str[-2]
        load_excel_files_df['Classify'] = ad_type
        load_excel_files_df['Imp'] = load_excel_files_df['Impressions']
        load_excel_files_df['Total Sales'] = load_excel_files_df['14 Day Total Sales '].round(2)
        load_excel_files_df['SKU Units'] = 0
        load_excel_files_df['SKU Sales'] = 0
        load_excel_files_df['Total Orders'] = load_excel_files_df['14 Day Total Orders (#)'] 

    if AD_TYPE_SP is ad_type:
        load_excel_files_df = load_excel_files_df[load_excel_files_df['Advertised SKU'].notnull()]
        load_excel_files_df['Year'] = load_excel_files_df['Date'].dt.year
        load_excel_files_df['Month'] = load_excel_files_df['Date'].dt.month
        load_excel_files_df['Week'] = load_excel_files_df['Date'].apply(week_of_month)
        load_excel_files_df['Model'] = load_excel_files_df['Campaign Name'].apply(extract_sp_model)
        load_excel_files_df['Model_asin'] = load_excel_files_df.apply(lambda x: extract_sp_model_asin2(x['Advertised ASIN'], x['Advertised SKU']), axis=1)
        load_excel_files_df['Type'] = load_excel_files_df['Campaign Name'].str.split(pat="_").str[-2]
        load_excel_files_df['Classify'] = load_excel_files_df['Campaign Name'].apply(extract_sp_class1)
        load_excel_files_df['Imp'] = load_excel_files_df['Impressions']
        load_excel_files_df['Total Sales'] = load_excel_files_df['7 Day Total Sales '].round(2)
        load_excel_files_df['SKU Units'] = load_excel_files_df['7 Day Advertised SKU Units (#)']
        load_excel_files_df['SKU Sales'] = load_excel_files_df['7 Day Advertised SKU Sales ']
        load_excel_files_df['Total Orders'] = load_excel_files_df['7 Day Total Orders (#)']
    
    result_df = load_excel_files_df.filter(items=column_names)
    display(HTML(result_df.head(5).to_html()))
    return result_df
    
df_array = []
for ad_type,ad_file_path in file_dict.items():
    df = extract_excel(ad_type,ad_file_path)
    df_array.append(df)
    
result_df = pd.concat(df_array)
result_df['Date'] = result_df['Date'].dt.strftime('%Y/%m/%d')

print(result_df.info())
print(asin_not_found_set)
result_df.to_excel(r'F:\Amazon\SSPA\Campagin\row data\sspa_data.xlsx',index=False)
# 数据保存在本地

SB
Sponsored Brands Campaign report.xlsx


D:\app\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Date,Year,Month,Week,Model,Model_asin,Type,Classify,Campaign Name,Imp,Clicks,Spend,Total Sales,SKU Units,SKU Sales,Total Orders
0,2023-03-01,2023,3,1,S5Max,S5Max,AW,SBV,S5Max_KT_GS_AW_SBV,439,0,0.00,0.0,0,0,0
1,2023-03-02,2023,3,1,S5Max,S5Max,AW,SBV,S5Max_KT_GS_AW_SBV,1106,5,12.57,0.0,0,0,0
2,2023-03-03,2023,3,1,S5Max,S5Max,AW,SBV,S5Max_KT_GS_AW_SBV,100,0,0.00,0.0,0,0,0
3,2023-03-04,2023,3,1,S5Max,S5Max,AW,SBV,S5Max_KT_GS_AW_SBV,280,1,3.00,0.0,0,0,0
4,2023-03-05,2023,3,2,S5Max,S5Max,AW,SBV,S5Max_KT_GS_AW_SBV,963,2,4.41,0.0,0,0,0


SD
Sponsored Display Campaign report.xlsx


D:\app\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


...

SP
Sponsored Products Advertised product report.xlsx


D:\app\anaconda\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Date,Year,Month,Week,Model,Model_asin,Type,Classify,Campaign Name,Imp,Clicks,Spend,Total Sales,SKU Units,SKU Sales,Total Orders
0,2023-03-01,2023,3,1,Q7,S5Max,AW,SP,Q7_AT_All_AW_SP,7250,17,27.23,0.00,0,0.0,0
1,2023-03-02,2023,3,1,Q7,S5Max,AW,SP,Q7_AT_All_AW_SP,10046,32,56.57,578.97,0,0.0,3
2,2023-03-03,2023,3,1,Q7,S5Max,AW,SP,Q7_AT_All_AW_SP,7945,11,20.56,0.00,0,0.0,0
3,2023-03-04,2023,3,1,Q7,S5Max,AW,SP,Q7_AT_All_AW_SP,9046,17,30.35,0.00,0,0.0,0
4,2023-03-05,2023,3,2,Q7,S5Max,AW,SP,Q7_AT_All_AW_SP,11334,15,27.24,0.00,0,0.0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13999 entries, 0 to 11134
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           13999 non-null  object 
 1   Year           13999 non-null  int64  
 2   Month          13999 non-null  int64  
 3   Week           13999 non-null  int64  
 4   Model          13999 non-null  object 
 5   Model_asin     13999 non-null  object 
 6   Type           13999 non-null  object 
 7   Classify       13999 non-null  object 
 8   Campaign Name  13999 non-null  object 
 9   Imp            13999 non-null  int64  
 10  Clicks         13999 non-null  int64  
 11  Spend          13999 non-null  float64
 12  Total Sales    13999 non-null  float64
 13  SKU Units      13999 non-null  int64  
 14  SKU Sales      13999 non-null  float64
 15  Total Orders   13999 non-null  int64  
dtypes: float64(3), int64(7), object(6)
memory usage: 1.8+ MB
None
set()


<ipython-input-8-d213267ba608>:144: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  result_df.to_excel(r'F:\Amazon\SSPA\Campagin\row data\sspa_data.xlsx',index=False)


In [9]:
# import re
# test = 'S7-E5mop-S7Maxv ultra_KT_iRobot_CD_SBV-mop'
# test1 = re.search('SB-?(?![A-Z])', test)

# print(test1)